<a href="https://colab.research.google.com/github/R4HUL-ROY/Multimodal_Results/blob/main/Multimodal_FinalCode.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install tensorflow==2.8
!apt install --allow-change-held-packages libcudnn8=8.1.0.77-1+cuda11.2

In [ ]:
# !pip install -q tensorflow_text
# !pip install fasttext
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import os
import pathlib
import tensorflow as tf
import fasttext.util

# import tensorflow_hub as hub
# import tensorflow_text as text
from tensorflow import keras
from tensorflow.keras import regularizers
from tensorflow.keras.models import Model, load_model
from tensorflow.keras.optimizers import Adam, Adamax

In [ ]:
merged_dataset_path = "/home/PiyaliBhunia/Desktop/GPU_shared_data/Resources/dataset/merged_data_for_GPU.csv"
stopwords_path = "/home/PiyaliBhunia/Desktop/GPU_shared_data/Resources/stopwords/stopwords.txt"
glove_vector_path = '/home/PiyaliBhunia/Desktop/GPU_shared_data/Resources/glove_vectors/glove.6B.200d.txt'
fasttext_model_path = '/home/PiyaliBhunia/Desktop/GPU_shared_data/Resources/fasttext_model/cc.en.300.bin'

vgg16_imagenet_weight_path = "/home/PiyaliBhunia/Desktop/GPU_shared_data/Resources/imagenet_weights/vgg16_weights_tf_dim_ordering_tf_kernels_notop.h5"
resnet50_imagenet_weight_path = "/home/PiyaliBhunia/Desktop/GPU_shared_data/Resources/imagenet_weights/resnet50_weights_tf_dim_ordering_tf_kernels_notop.h5"
mobilenetv2_imagenet_weight_path = "/home/PiyaliBhunia/Desktop/GPU_shared_data/Resources/imagenet_weights/mobilenet_v2_weights_tf_dim_ordering_tf_kernels_1.0_224_no_top.h5"

# Read the data

In [ ]:
img_data_root = pathlib.Path('/home/PiyaliBhunia/Desktop/GPU_shared_data/Resources/dataset/Tobacco3482-jpg')
print(img_data_root)
for item in img_data_root.iterdir():
    print(item)

print()

text_data_root = pathlib.Path('/home/PiyaliBhunia/Desktop/GPU_shared_data/Resources/dataset/tobaco_OCR')
print(text_data_root)
for item in text_data_root.iterdir():
    print(item)

/home/PiyaliBhunia/Desktop/GPU_shared_data/Resources/dataset/Tobacco3482-jpg
/home/PiyaliBhunia/Desktop/GPU_shared_data/Resources/dataset/Tobacco3482-jpg/ADVE
/home/PiyaliBhunia/Desktop/GPU_shared_data/Resources/dataset/Tobacco3482-jpg/Email
/home/PiyaliBhunia/Desktop/GPU_shared_data/Resources/dataset/Tobacco3482-jpg/Form
/home/PiyaliBhunia/Desktop/GPU_shared_data/Resources/dataset/Tobacco3482-jpg/Letter
/home/PiyaliBhunia/Desktop/GPU_shared_data/Resources/dataset/Tobacco3482-jpg/Memo
/home/PiyaliBhunia/Desktop/GPU_shared_data/Resources/dataset/Tobacco3482-jpg/News
/home/PiyaliBhunia/Desktop/GPU_shared_data/Resources/dataset/Tobacco3482-jpg/Note
/home/PiyaliBhunia/Desktop/GPU_shared_data/Resources/dataset/Tobacco3482-jpg/Report
/home/PiyaliBhunia/Desktop/GPU_shared_data/Resources/dataset/Tobacco3482-jpg/Resume
/home/PiyaliBhunia/Desktop/GPU_shared_data/Resources/dataset/Tobacco3482-jpg/Scientific

/home/PiyaliBhunia/Desktop/GPU_shared_data/Resources/dataset/tobaco_OCR
/home/PiyaliBhuni

In [ ]:
def get_corresponding_txtpath(img_path):
    return img_path.replace("Tobacco3482-jpg", "tobaco_OCR")[:-3] + "txt"
    

def get_file_paths_and_labels(img_data_root, text_data_root):
    img_paths = [str(path) for path in img_data_root.glob('*/*.jpg')]
    text_paths = [get_corresponding_txtpath(this_path) for this_path in img_paths]
    img_labels = [p.split("/")[-2] for p in img_paths]
    text_labels = [p.split("/")[-2] for p in text_paths]
    return img_paths, img_labels, text_paths, text_labels

img_paths, img_labels, text_paths, text_labels = get_file_paths_and_labels(img_data_root, text_data_root)
print(len(img_paths))
print(len(img_labels))
print(len(text_paths))
print(len(text_labels))

3482
3482
3482
3482


In [ ]:
def get_text_from_path(path):
    with open(path) as f:
        lines = f.readlines()
        lines  = ' '.join(lines)
        f.close()
    return lines

In [ ]:
text_contains = []
for i, this_path in enumerate(text_paths):
    text_contains.append(get_text_from_path(this_path))


In [ ]:
df = pd.DataFrame(list(zip(text_paths, text_contains, img_paths, img_labels)),
               columns =['text_paths','texts', 'img_paths', 'data_label'])

# Merged Dataframe
df.head()

,text_paths,texts,img_paths,data_label
0,/home/PiyaliBhunia/Desktop/GPU_shared_data/Res...,| O49 398499\n Ne\n Chait\n q catty ¥ iM\n,/home/PiyaliBhunia/Desktop/GPU_shared_data/Res...,ADVE
1,/home/PiyaliBhunia/Desktop/GPU_shared_data/Res...,\n \n \n \n A Mpertant as yar\n sesiye teaeter...,/home/PiyaliBhunia/Desktop/GPU_shared_data/Res...,ADVE
2,/home/PiyaliBhunia/Desktop/GPU_shared_data/Res...,,/home/PiyaliBhunia/Desktop/GPU_shared_data/Res...,ADVE
3,/home/PiyaliBhunia/Desktop/GPU_shared_data/Res...,\n TE che fitm\n m66400 7127\n KOOLS are the o...,/home/PiyaliBhunia/Desktop/GPU_shared_data/Res...,ADVE
4,/home/PiyaliBhunia/Desktop/GPU_shared_data/Res...,"LEO BE STV ELSA,\n Te CMON INN\n \n \n \n AW N...",/home/PiyaliBhunia/Desktop/GPU_shared_data/Res...,ADVE


In [ ]:
# df.to_csv("merged_data_for_multimodal_model.csv")

# Read dataset from csv

In [ ]:
# df = pd.read_csv(merged_dataset_path)
# print(len(df))
# df.head()

In [ ]:
from sklearn import preprocessing
le = preprocessing.LabelEncoder()
df['data_label']= le.fit_transform(df['data_label'])
df.head()

,text_paths,texts,img_paths,data_label
0,/home/PiyaliBhunia/Desktop/GPU_shared_data/Res...,| O49 398499\n Ne\n Chait\n q catty ¥ iM\n,/home/PiyaliBhunia/Desktop/GPU_shared_data/Res...,0
1,/home/PiyaliBhunia/Desktop/GPU_shared_data/Res...,\n \n \n \n A Mpertant as yar\n sesiye teaeter...,/home/PiyaliBhunia/Desktop/GPU_shared_data/Res...,0
2,/home/PiyaliBhunia/Desktop/GPU_shared_data/Res...,,/home/PiyaliBhunia/Desktop/GPU_shared_data/Res...,0
3,/home/PiyaliBhunia/Desktop/GPU_shared_data/Res...,\n TE che fitm\n m66400 7127\n KOOLS are the o...,/home/PiyaliBhunia/Desktop/GPU_shared_data/Res...,0
4,/home/PiyaliBhunia/Desktop/GPU_shared_data/Res...,"LEO BE STV ELSA,\n Te CMON INN\n \n \n \n AW N...",/home/PiyaliBhunia/Desktop/GPU_shared_data/Res...,0


In [ ]:
label_dict = {}
for idx, row in enumerate(df['text_paths']):
    path = df.at[idx, 'text_paths']
    lab = path.split("/")[-2]
    label_dict[str(lab)] = df.at[idx, 'data_label']
    
print(label_dict) 

{'ADVE': 0, 'Email': 1, 'Form': 2, 'Letter': 3, 'Memo': 4, 'News': 5, 'Note': 6, 'Report': 7, 'Resume': 8, 'Scientific': 9}


In [ ]:
import re
def preprocess(text_string):
    preprocessed_string = re.sub(r'[^\w\s]','',text_string)
    preprocessed_string = preprocessed_string.replace('\n',' ')
    preprocessed_string = preprocessed_string.replace('_',' ')
    preprocessed_string = re.sub(' +', ' ', preprocessed_string)
    return preprocessed_string

In [ ]:
# Tokenize, Lemmatize, stopwords removal
import spacy 
import nltk
nlp = spacy.load("en_core_web_sm")

def get_stopwords(file_path):
    with open(file_path, "r") as fp:
        content = fp.read()
        stops = content.split("\n")
        stops = stops[:-1]
        fp.close()
        return stops

stops = get_stopwords(stopwords_path)
def normalize(comment, lowercase, remove_stopwords):
    if lowercase:
        comment = comment.lower()
    comment = nlp(comment)
    lemmatized = list()
    for word in comment:
        lemma = word.lemma_.strip()
        if lemma:
            if not remove_stopwords or (remove_stopwords and lemma not in stops):
                lemmatized.append(lemma)
    return " ".join(lemmatized)

normalize("counting playing the Home", lowercase=True, remove_stopwords=True)

'count play home'

In [ ]:
df['texts'] = [preprocess(str(this_text)) for this_text in df['texts']]
df['texts'] = [normalize(this_text, lowercase=True, remove_stopwords=True) for this_text in df['texts']]
df.head()

,text_paths,texts,img_paths,data_label
0,/home/PiyaliBhunia/Desktop/GPU_shared_data/Res...,o49 398499 ne chait q catty I,/home/PiyaliBhunia/Desktop/GPU_shared_data/Res...,0
1,/home/PiyaliBhunia/Desktop/GPU_shared_data/Res...,mpertant yar sesiye teaetere cabiieess baely k...,/home/PiyaliBhunia/Desktop/GPU_shared_data/Res...,0
2,/home/PiyaliBhunia/Desktop/GPU_shared_data/Res...,,/home/PiyaliBhunia/Desktop/GPU_shared_data/Res...,0
3,/home/PiyaliBhunia/Desktop/GPU_shared_data/Res...,te che fitm m66400 7127 kool cigarette taste g...,/home/PiyaliBhunia/Desktop/GPU_shared_data/Res...,0
4,/home/PiyaliBhunia/Desktop/GPU_shared_data/Res...,leo stv elsa te cmon inn aw na 338 tigre lilly...,/home/PiyaliBhunia/Desktop/GPU_shared_data/Res...,0


In [ ]:
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

In [ ]:
def dataframe_to_dataset(dataframe):
    d = {}
    embed_tensor = []
    for i in dataframe['texts_embedding']:
        embed_tensor.append(tf.convert_to_tensor(i))

    img_path_tensor= []
    for i in dataframe['img_paths']:
        img_path_tensor.append(tf.convert_to_tensor(i))

    d['texts_embedding'] = embed_tensor
    d['img_paths'] = img_path_tensor

    labels = dataframe["data_label"]
    labels = tf.convert_to_tensor(labels)
    ds = tf.data.Dataset.from_tensor_slices((d, labels))
    return ds

In [ ]:
@tf.function
def preprocess_image(image_path):
    extension = tf.strings.split(image_path)[-1]

    image = tf.io.read_file(image_path)
    if extension == b"jpg":
        image = tf.image.decode_jpeg(image, 3)
    else:
        image = tf.image.decode_png(image, 3)
    image = tf.image.resize(image, (224, 224))
    image = tf.image.per_image_standardization(image)
    return image

@tf.function
def preprocess_text(sample):
    pass

@tf.function
def preprocess_text_and_image(sample):
    image = preprocess_image(sample["img_paths"])
    text = sample['texts_embedding']
    return {"image_inputs": image,  "text_inputs": text}

In [ ]:
batch_size = 64
auto = tf.data.experimental.AUTOTUNE

def prepare_dataset(dataframe, training = True):
    ds = dataframe_to_dataset(dataframe)
    if training:
        ds = ds.shuffle(len(train_df))
    ds = ds.map(lambda x, y: (preprocess_text_and_image(x), y)).cache()
    ds = ds.batch(batch_size).prefetch(auto)
    return ds

In [ ]:
from keras.models import Sequential
from keras.layers import Embedding,  Flatten,  Dense, Dropout, Conv1D, GlobalMaxPooling1D, MaxPooling1D, GlobalMaxPooling2D
from keras.initializers import Constant
from tensorflow.keras.optimizers import Adam

In [ ]:
def get_embedding_matrix(text_model):
    if text_model == "glove":
        emmbed_dict = {}
        with open(glove_vector_path ,'r') as f:
            for line in f:
                values = line.split()
                word = values[0]
                vector = np.asarray(values[1:],'float32')
                emmbed_dict[word]=vector
            f.close()
        embedding_matrix = np.zeros((num_words, 200))
        for word, i in word_index.items():
            if i < num_words:
                emb_vec = emmbed_dict.get(word)
                if emb_vec is not None:
                    embedding_matrix[i] = emb_vec
        return embedding_matrix 

    elif text_model == "fasttext":  
        ft = fasttext.load_model(fasttext_model_path)    
        embedding_matrix = np.zeros((num_words, 300)) 
        for word, i in word_index.items():
            if i < num_words:
                emb_vec = ft.get_word_vector(word)
                if emb_vec is not None:
                    embedding_matrix[i] = emb_vec 
        return embedding_matrix    
    else:    
        return None                                       

In [ ]:
def create_text_encoder(text_projection_dims, dropout_rate, text_model, text_model_trainable):

    embedding_matrix = get_embedding_matrix(text_model)

    filter_size = 128
    stride_unit = 1
    kernels = 3
    dropout_rate = 0.2
    dropout_rate_conv = 0.5

    inputs = keras.Input(shape=(500,), dtype=tf.int32, name="text_inputs")

    if text_model == "glove":
        embed = tf.keras.layers.Embedding(  num_words,
                                            200,
                                            input_length = 500,
                                            embeddings_initializer = Constant(embedding_matrix),
                                            trainable = text_model_trainable
                                        )(inputs)
    if text_model == "fasttext":
        embed = tf.keras.layers.Embedding(  num_words,
                                            300,
                                            input_length = 500,
                                            embeddings_initializer = Constant(embedding_matrix),
                                            trainable = text_model_trainable
                                        )(inputs)

    x = tf.keras.layers.Dropout(dropout_rate)(embed)


    x = tf.keras.layers.Conv1D(filters=filter_size,kernel_size=kernels, padding='valid',activation='relu',strides=stride_unit,
                               kernel_initializer = "glorot_uniform" ,bias_initializer = 'zeros')(x)
    x = tf.keras.layers.MaxPooling1D()(x)
    x = tf.keras.layers.Dropout(dropout_rate_conv)(x)

    x = tf.keras.layers.Conv1D(filters=filter_size,kernel_size=kernels, padding='valid',activation='relu',strides=stride_unit,
                               kernel_initializer = "glorot_uniform" ,bias_initializer = 'zeros')(x)
    x = tf.keras.layers.MaxPooling1D()(x)
    x = tf.keras.layers.Dropout(dropout_rate_conv)(x)

    x = tf.keras.layers.Conv1D(filters=filter_size,kernel_size=kernels, padding='valid',activation='relu',strides=stride_unit,
                               kernel_initializer = "glorot_uniform" ,bias_initializer = 'zeros')(x)
    x = tf.keras.layers.MaxPooling1D()(x)
    x = tf.keras.layers.Dropout(dropout_rate_conv)(x)
    
    x = tf.keras.layers.Conv1D(filters=filter_size,kernel_size=kernels, padding='valid',activation='relu',strides=stride_unit,
                               kernel_initializer = "glorot_uniform" ,bias_initializer = 'zeros')(x)
    x = tf.keras.layers.MaxPooling1D()(x)
    x = tf.keras.layers.Dropout(dropout_rate_conv)(x)

    x = tf.keras.layers.Flatten()(x)
    x = tf.keras.layers.Dropout(dropout_rate_conv)(x)
     
    outputs = keras.layers.Dense(units=text_projection_dims,activation="relu",kernel_initializer="glorot_uniform",bias_initializer='zeros')(x)
    return keras.Model(inputs, outputs, name="text_encoder")

In [ ]:
def create_vision_encoder(img_projection_dims, dropout_rate, image_model, img_base_model_trainable):
    img_shape=(224,224,3)
    seed_value = 42


    if image_model == "vgg16":
        base_model = tf.keras.applications.vgg16.VGG16(include_top=False,input_shape=img_shape,pooling='avg',classes=10,weights=None)
        base_model.load_weights(vgg16_imagenet_weight_path)
        base_model.trainable = img_base_model_trainable

    elif image_model == "resnet50":
        base_model= tf.keras.applications.ResNet50(include_top=False,input_shape=img_shape,pooling='avg',classes=10,weights=None)
        base_model.load_weights(resnet50_imagenet_weight_path)
        base_model.trainable = img_base_model_trainable

    elif image_model == "mobilenetv2":
        base_model=tf.keras.applications.MobileNetV2(include_top=False,input_shape=img_shape,pooling='avg', classes=10,weights=None)
        base_model.load_weights(mobilenetv2_imagenet_weight_path)
        base_model.trainable = img_base_model_trainable


    inputs = keras.Input(shape=img_shape)
    x = base_model(inputs, training = img_base_model_trainable)
    outputs =tf.keras.layers.Dense(img_projection_dims,kernel_initializer ="glorot_uniform",bias_initializer= "zeros",activation='relu')(x)

    model=Model(inputs, outputs)

    # Receive the images as inputs.
    image_inputs = keras.Input(shape=(224, 224, 3), name="image_inputs")

    if image_model == "vgg16":
        preprocessed_image = tf.keras.applications.vgg16.preprocess_input(image_inputs)
    if image_model == "resnet50":
        preprocessed_image = tf.keras.applications.resnet50.preprocess_input(image_inputs)    
    if image_model == "mobilenetv2":
        preprocessed_image = tf.keras.applications.mobilenet_v2.preprocess_input(image_inputs)

    embeddings = model(preprocessed_image)

    # Create the vision encoder model.
    return keras.Model(image_inputs, embeddings, name="vision_encoder")

In [ ]:
def create_multimodal_model(**kwargs):

    img_projection_dims = kwargs['img_projection_dims']
    text_projection_dims = kwargs['text_projection_dims']
    dropout_rate = kwargs['dropout_rate']
    image_model = kwargs['image_model']
    text_model = kwargs['text_model']
    img_base_model_trainable = kwargs['img_base_model_trainable']
    text_model_trainable = kwargs['text_model_trainable']

    if image_model == "None" and text_model == "None":
        print("Both the image_model and text_model cannot be None at the same time !!")
        return
    # Receive the images and text as inputs.
    if image_model != "None":
        image_inputs = keras.Input(shape=(224, 224, 3), name="image_inputs")
        vision_encoder = create_vision_encoder(img_projection_dims, dropout_rate, image_model, img_base_model_trainable)
        vision_projections = vision_encoder(image_inputs)

    if text_model != "None":   
        text_inputs = keras.Input(shape=(500,), dtype=tf.int32, name="text_inputs")
        text_encoder = create_text_encoder(text_projection_dims, dropout_rate, text_model, text_model_trainable)
        text_projections = text_encoder(text_inputs)


    # Concatenate the projections and pass through the classification layer.
    if image_model != "None" and text_model != "None":
        concatenated = keras.layers.Concatenate()([vision_projections, text_projections])
    # concatenated = tf.keras.layers.BatchNormalization(momentum=0.9)(concatenated)


    if image_model == "None" and text_model != "None":
        outputs = keras.layers.Dense(10, activation="softmax")(text_projections)
        return keras.Model(text_inputs, outputs)
    elif text_model == "None" and image_model != "None":    
        outputs = keras.layers.Dense(10, activation="softmax")(vision_projections)
        return keras.Model(image_inputs, outputs)
    elif text_model != "None" and image_model != "None":
        outputs = keras.layers.Dense(10, activation="softmax")(concatenated)
        return keras.Model([image_inputs, text_inputs], outputs)
    else:
        return None

    

In [ ]:
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.metrics import precision_score, recall_score, f1_score, accuracy_score
import random
random.seed(56)

In [ ]:
epochs = 50
# fine_tune_epochs = 5
lr = 0.001

d_glove = {"precision" : [],"recall" : [],"f1_score" : [],"accuracy" : [],"seed_value" : []}
d_fasttext = {"precision" : [],"recall" : [],"f1_score" : [],"accuracy" : [],"seed_value" : []}

d_vgg16 = {"precision" : [],"recall" : [],"f1_score" : [],"accuracy" : [],"seed_value" : []}
d_mobilenetv2 = {"precision" : [],"recall" : [],"f1_score" : [],"accuracy" : [],"seed_value" : []}
d_resnet50 = {"precision" : [],"recall" : [],"f1_score" : [],"accuracy" : [],"seed_value" : []}

d_glove_vgg16 = {"precision" : [],"recall" : [],"f1_score" : [],"accuracy" : [],"seed_value" : []}
d_glove_mobilenetv2 = {"precision" : [],"recall" : [],"f1_score" : [],"accuracy" : [],"seed_value" : []}
d_glove_resnet50 = {"precision" : [],"recall" : [],"f1_score" : [],"accuracy" : [],"seed_value" : []}

d_fasttext_vgg16 = {"precision" : [],"recall" : [],"f1_score" : [],"accuracy" : [],"seed_value" : []}
d_fasttext_mobilenetv2 = {"precision" : [],"recall" : [],"f1_score" : [],"accuracy" : [],"seed_value" : []}
d_fasttext_resnet50 = {"precision" : [],"recall" : [],"f1_score" : [],"accuracy" : [],"seed_value" : []}

seed = []

for i in range(10):
    print('#Run: ', i+1)
    sd = random.randint(0, 100)
    seed.append(sd)

    # Split the dataframe
    train_df, test_df = train_test_split(df, test_size=0.2, random_state=sd)
    test_df, val_df = train_test_split(test_df, test_size=0.5, random_state=sd)

    max_len = 500
    # Fit the tokenizer
    tokenizer = Tokenizer(num_words=65000)
    tokenizer.fit_on_texts(train_df['texts'])

    # sequence the input corpus and add zero padding upto 500 word
    train_sequence = tokenizer.texts_to_sequences(train_df['texts'])
    train_padded = pad_sequences(train_sequence, maxlen = max_len, truncating = "post", padding = "post" )

    valid_sequence = tokenizer.texts_to_sequences(val_df['texts'])
    valid_padded = pad_sequences(valid_sequence, maxlen = max_len, truncating = "post", padding = "post" )

    test_sequence = tokenizer.texts_to_sequences(test_df['texts'])
    test_padded = pad_sequences(test_sequence, maxlen = max_len, truncating = "post", padding = "post" )

    train_tensor = [tf.convert_to_tensor(train_padded[i]) for i in range(train_padded.shape[0])]
    train_df['texts_embedding'] = train_tensor

    test_tensor = [tf.convert_to_tensor(test_padded[i]) for i in range(test_padded.shape[0])]
    test_df['texts_embedding'] = test_tensor   

    val_tensor = [tf.convert_to_tensor(valid_padded[i]) for i in range(valid_padded.shape[0])]
    val_df['texts_embedding'] = val_tensor

    # Preparing dataset
    train_ds = prepare_dataset(train_df)
    val_ds = prepare_dataset(val_df, False)
    test_ds = prepare_dataset(test_df, False)

    word_index = tokenizer.word_index
    num_words = len(word_index) + 1

    """
    options for image_model : ["vgg16", "resnet50", "mobilenetv2"] (case-sensitive input)
    options for text_model : ["glove", "fasttext"] (case-sensitive input)
    """
    
    ## VGG16 model
    vgg16_model = create_multimodal_model( 
        img_projection_dims=512,
        text_projection_dims=512,
        dropout_rate=0.2,
        image_model = "vgg16",
        text_model = "None", 
        img_base_model_trainable=False,
        text_model_trainable= False
    )

    vgg16_model.compile(Adamax(learning_rate=0.001), loss="sparse_categorical_crossentropy", metrics=["accuracy"])
    vgg16_history = vgg16_model.fit(train_ds, validation_data=val_ds, epochs=epochs, verbose=2)
    
    y_pred = vgg16_model.predict(test_ds)
    y_pred = [np.argmax(i) for i in y_pred]

    y_true = []
    for element in test_ds:
        y_true.extend(list(element[1].numpy()))
    
    d_vgg16["precision"].append(float(precision_score(y_true, y_pred, average='weighted')))
    d_vgg16["recall"].append(float(recall_score(y_true, y_pred, average='weighted')))
    d_vgg16["f1_score"].append(float(f1_score(y_true, y_pred, average='weighted')))
    d_vgg16['accuracy'].append(float(accuracy_score(y_true, y_pred)))
    
    
    ### Resnet 50 model
    resnet50_model = create_multimodal_model( 
        img_projection_dims=512,
        text_projection_dims=512,
        dropout_rate=0.2,
        image_model = "resnet50",
        text_model = "None", 
        img_base_model_trainable=False,
        text_model_trainable= False
    )

    resnet50_model.compile(Adamax(learning_rate=0.001), loss="sparse_categorical_crossentropy", metrics=["accuracy"])
    resnet50_history = resnet50_model.fit(train_ds, validation_data=val_ds, epochs=epochs, verbose=2)
    
    y_pred = resnet50_model.predict(test_ds)
    y_pred = [np.argmax(i) for i in y_pred]

    y_true = []
    for element in test_ds:
        y_true.extend(list(element[1].numpy()))
    
    d_resnet50["precision"].append(float(precision_score(y_true, y_pred, average='weighted')))
    d_resnet50["recall"].append(float(recall_score(y_true, y_pred, average='weighted')))
    d_resnet50["f1_score"].append(float(f1_score(y_true, y_pred, average='weighted')))
    d_resnet50['accuracy'].append(float(accuracy_score(y_true, y_pred)))
    
    
    # mobilenetv2 model
    mobilenetv2_model = create_multimodal_model( 
        img_projection_dims=512,
        text_projection_dims=512,
        dropout_rate=0.2,
        image_model = "mobilenetv2",
        text_model = "None", 
        img_base_model_trainable=False,
        text_model_trainable= False
    )

    mobilenetv2_model.compile(Adamax(learning_rate=0.001), loss="sparse_categorical_crossentropy", metrics=["accuracy"])
    mobilenetv2_history = mobilenetv2_model.fit(train_ds, validation_data=val_ds, epochs=epochs, verbose=2)
    
    y_pred = mobilenetv2_model.predict(test_ds)
    y_pred = [np.argmax(i) for i in y_pred]

    y_true = []
    for element in test_ds:
        y_true.extend(list(element[1].numpy()))
    
    d_mobilenetv2["precision"].append(float(precision_score(y_true, y_pred, average='weighted')))
    d_mobilenetv2["recall"].append(float(recall_score(y_true, y_pred, average='weighted')))
    d_mobilenetv2["f1_score"].append(float(f1_score(y_true, y_pred, average='weighted')))
    d_mobilenetv2['accuracy'].append(float(accuracy_score(y_true, y_pred)))
    
    
    
    ## Glove model
    glove_model = create_multimodal_model( 
        img_projection_dims=512,
        text_projection_dims=512,
        dropout_rate=0.2,
        image_model = "None",
        text_model = "glove", 
        img_base_model_trainable=False,
        text_model_trainable= False
    )

    glove_model.compile(Adamax(learning_rate=0.001), loss="sparse_categorical_crossentropy", metrics=["accuracy"])
    glove_history = glove_model.fit(train_ds, validation_data=val_ds, epochs=epochs, verbose=2)
    
    y_pred = glove_model.predict(test_ds)
    y_pred = [np.argmax(i) for i in y_pred]

    y_true = []
    for element in test_ds:
        y_true.extend(list(element[1].numpy()))
    
    d_glove["precision"].append(float(precision_score(y_true, y_pred, average='weighted')))
    d_glove["recall"].append(float(recall_score(y_true, y_pred, average='weighted')))
    d_glove["f1_score"].append(float(f1_score(y_true, y_pred, average='weighted')))
    d_glove['accuracy'].append(float(accuracy_score(y_true, y_pred)))
    
    
    
    ## Fasttext model
    fasttext_model = create_multimodal_model( 
        img_projection_dims=512,
        text_projection_dims=512,
        dropout_rate=0.2,
        image_model = "None",
        text_model = "fasttext", 
        img_base_model_trainable=False,
        text_model_trainable= False
    )

    fasttext_model.compile(Adamax(learning_rate=0.001), loss="sparse_categorical_crossentropy", metrics=["accuracy"])
    fasttext_history = fasttext_model.fit(train_ds, validation_data=val_ds, epochs=epochs, verbose=2)
    
    y_pred = fasttext_model.predict(test_ds)
    y_pred = [np.argmax(i) for i in y_pred]

    y_true = []
    for element in test_ds:
        y_true.extend(list(element[1].numpy()))
    
    d_fasttext["precision"].append(float(precision_score(y_true, y_pred, average='weighted')))
    d_fasttext["recall"].append(float(recall_score(y_true, y_pred, average='weighted')))
    d_fasttext["f1_score"].append(float(f1_score(y_true, y_pred, average='weighted')))
    d_fasttext['accuracy'].append(float(accuracy_score(y_true, y_pred)))
    
    
    
    
    ## Glove + vgg16
    glove_vgg16_model = create_multimodal_model( 
        img_projection_dims=512,
        text_projection_dims=512,
        dropout_rate=0.2,
        image_model = "vgg16",
        text_model = "glove", 
        img_base_model_trainable=False,
        text_model_trainable= False
    )

    glove_vgg16_model.compile(Adamax(learning_rate=0.001), loss="sparse_categorical_crossentropy", metrics=["accuracy"])
    glove_vgg16_history = glove_vgg16_model.fit(train_ds, validation_data=val_ds, epochs=epochs, verbose=2)
    
    y_pred = glove_vgg16_model.predict(test_ds)
    y_pred = [np.argmax(i) for i in y_pred]

    y_true = []
    for element in test_ds:
        y_true.extend(list(element[1].numpy()))
    
    d_glove_vgg16["precision"].append(float(precision_score(y_true, y_pred, average='weighted')))
    d_glove_vgg16["recall"].append(float(recall_score(y_true, y_pred, average='weighted')))
    d_glove_vgg16["f1_score"].append(float(f1_score(y_true, y_pred, average='weighted')))
    d_glove_vgg16['accuracy'].append(float(accuracy_score(y_true, y_pred)))
    
    
    ## glove + resnet50
    glove_resnet50_model = create_multimodal_model( 
        img_projection_dims=512,
        text_projection_dims=512,
        dropout_rate=0.2,
        image_model = "resnet50",
        text_model = "glove", 
        img_base_model_trainable=False,
        text_model_trainable= False
    )

    glove_resnet50_model.compile(Adamax(learning_rate=0.001), loss="sparse_categorical_crossentropy", metrics=["accuracy"])
    glove_resnet50_history = glove_resnet50_model.fit(train_ds, validation_data=val_ds, epochs=epochs, verbose=2)
    
    y_pred = glove_resnet50_model.predict(test_ds)
    y_pred = [np.argmax(i) for i in y_pred]

    y_true = []
    for element in test_ds:
        y_true.extend(list(element[1].numpy()))
    
    d_glove_resnet50["precision"].append(float(precision_score(y_true, y_pred, average='weighted')))
    d_glove_resnet50["recall"].append(float(recall_score(y_true, y_pred, average='weighted')))
    d_glove_resnet50["f1_score"].append(float(f1_score(y_true, y_pred, average='weighted')))
    d_glove_resnet50['accuracy'].append(float(accuracy_score(y_true, y_pred)))
    
    
    ## glove + mobilenetv2
    glove_mobilenetv2_model = create_multimodal_model( 
        img_projection_dims=512,
        text_projection_dims=512,
        dropout_rate=0.2,
        image_model = "mobilenetv2",
        text_model = "glove", 
        img_base_model_trainable=False,
        text_model_trainable= False
    )

    glove_mobilenetv2_model.compile(Adamax(learning_rate=0.001), loss="sparse_categorical_crossentropy", metrics=["accuracy"])
    glove_mobilenetv2_history = glove_mobilenetv2_model.fit(train_ds, validation_data=val_ds, epochs=epochs, verbose=2)
    
    y_pred = glove_mobilenetv2_model.predict(test_ds)
    y_pred = [np.argmax(i) for i in y_pred]

    y_true = []
    for element in test_ds:
        y_true.extend(list(element[1].numpy()))
    
    d_glove_mobilenetv2["precision"].append(float(precision_score(y_true, y_pred, average='weighted')))
    d_glove_mobilenetv2["recall"].append(float(recall_score(y_true, y_pred, average='weighted')))
    d_glove_mobilenetv2["f1_score"].append(float(f1_score(y_true, y_pred, average='weighted')))
    d_glove_mobilenetv2['accuracy'].append(float(accuracy_score(y_true, y_pred)))
    
    
    ## fasttext + vgg16
    fasttext_vgg16_model = create_multimodal_model( 
        img_projection_dims=512,
        text_projection_dims=512,
        dropout_rate=0.2,
        image_model = "vgg16",
        text_model = "fasttext", 
        img_base_model_trainable=False,
        text_model_trainable= False
    )

    fasttext_vgg16_model.compile(Adamax(learning_rate=0.001), loss="sparse_categorical_crossentropy", metrics=["accuracy"])
    fasttext_vgg16_history = fasttext_vgg16_model.fit(train_ds, validation_data=val_ds, epochs=epochs, verbose=2)
    
    y_pred = fasttext_vgg16_model.predict(test_ds)
    y_pred = [np.argmax(i) for i in y_pred]

    y_true = []
    for element in test_ds:
        y_true.extend(list(element[1].numpy()))
    
    d_fasttext_vgg16["precision"].append(float(precision_score(y_true, y_pred, average='weighted')))
    d_fasttext_vgg16["recall"].append(float(recall_score(y_true, y_pred, average='weighted')))
    d_fasttext_vgg16["f1_score"].append(float(f1_score(y_true, y_pred, average='weighted')))
    d_fasttext_vgg16['accuracy'].append(float(accuracy_score(y_true, y_pred)))
    
    
    ## fasttext + resnet50
    fasttext_resnet50_model = create_multimodal_model( 
        img_projection_dims=512,
        text_projection_dims=512,
        dropout_rate=0.2,
        image_model = "resnet50",
        text_model = "fasttext", 
        img_base_model_trainable=False,
        text_model_trainable= False
    )

    fasttext_resnet50_model.compile(Adamax(learning_rate=0.001), loss="sparse_categorical_crossentropy", metrics=["accuracy"])
    fasttext_resnet50_history = fasttext_resnet50_model.fit(train_ds, validation_data=val_ds, epochs=epochs, verbose=2)
    
    y_pred = fasttext_resnet50_model.predict(test_ds)
    y_pred = [np.argmax(i) for i in y_pred]

    y_true = []
    for element in test_ds:
        y_true.extend(list(element[1].numpy()))
    
    d_fasttext_resnet50["precision"].append(float(precision_score(y_true, y_pred, average='weighted')))
    
    d_fasttext_resnet50["recall"].append(float(recall_score(y_true, y_pred, average='weighted')))
    d_fasttext_resnet50["f1_score"].append(float(f1_score(y_true, y_pred, average='weighted')))
    d_fasttext_resnet50['accuracy'].append(float(accuracy_score(y_true, y_pred)))
    
    
    ## fasttext + mobilenetv2
    fasttext_mobilenetv2_model = create_multimodal_model( 
        img_projection_dims=512,
        text_projection_dims=512,
        dropout_rate=0.2,
        image_model = "mobilenetv2",
        text_model = "fasttext", 
        img_base_model_trainable=False,
        text_model_trainable= False
    )

    fasttext_mobilenetv2_model.compile(Adamax(learning_rate=0.001), loss="sparse_categorical_crossentropy", metrics=["accuracy"])
    fasttext_mobilenetv2_history = fasttext_mobilenetv2_model.fit(train_ds, validation_data=val_ds, epochs=epochs, verbose=2)
    
    y_pred = fasttext_mobilenetv2_model.predict(test_ds)
    y_pred = [np.argmax(i) for i in y_pred]

    y_true = []
    for element in test_ds:
        y_true.extend(list(element[1].numpy()))
    
    d_fasttext_mobilenetv2["precision"].append(float(precision_score(y_true, y_pred, average='weighted')))
    d_fasttext_mobilenetv2["recall"].append(float(recall_score(y_true, y_pred, average='weighted')))
    d_fasttext_mobilenetv2["f1_score"].append(float(f1_score(y_true, y_pred, average='weighted')))
    d_fasttext_mobilenetv2['accuracy'].append(float(accuracy_score(y_true, y_pred)))

#Run:  1
Train for 44 steps, validate for 6 steps
Epoch 1/50
44/44 - 92s - loss: 1.8054 - accuracy: 0.3849 - val_loss: 1.5439 - val_accuracy: 0.4298
Epoch 2/50
44/44 - 6s - loss: 1.4199 - accuracy: 0.5260 - val_loss: 1.3512 - val_accuracy: 0.5530
Epoch 3/50
44/44 - 7s - loss: 1.2845 - accuracy: 0.5709 - val_loss: 1.2413 - val_accuracy: 0.5931
Epoch 4/50
44/44 - 7s - loss: 1.1968 - accuracy: 0.6079 - val_loss: 1.1698 - val_accuracy: 0.6132
Epoch 5/50
44/44 - 6s - loss: 1.1326 - accuracy: 0.6276 - val_loss: 1.1158 - val_accuracy: 0.6332
Epoch 6/50
44/44 - 7s - loss: 1.0819 - accuracy: 0.6478 - val_loss: 1.0750 - val_accuracy: 0.6361
Epoch 7/50
44/44 - 7s - loss: 1.0402 - accuracy: 0.6654 - val_loss: 1.0425 - val_accuracy: 0.6504
Epoch 8/50
44/44 - 6s - loss: 1.0052 - accuracy: 0.6779 - val_loss: 1.0155 - val_accuracy: 0.6648
Epoch 9/50
44/44 - 7s - loss: 0.9746 - accuracy: 0.6898 - val_loss: 0.9932 - val_accuracy: 0.6676
Epoch 10/50
44/44 - 6s - loss: 0.9475 - accuracy: 0.6937 - val_loss

44/44 - 5s - loss: 0.4003 - accuracy: 0.8700 - val_loss: 0.8103 - val_accuracy: 0.6905
Epoch 34/50
44/44 - 4s - loss: 0.3893 - accuracy: 0.8758 - val_loss: 0.8122 - val_accuracy: 0.6877
Epoch 35/50
44/44 - 5s - loss: 0.3786 - accuracy: 0.8822 - val_loss: 0.8165 - val_accuracy: 0.6877
Epoch 36/50
44/44 - 4s - loss: 0.3684 - accuracy: 0.8851 - val_loss: 0.8233 - val_accuracy: 0.6905
Epoch 37/50
44/44 - 5s - loss: 0.3582 - accuracy: 0.8908 - val_loss: 0.8279 - val_accuracy: 0.6905
Epoch 38/50
44/44 - 5s - loss: 0.3482 - accuracy: 0.8962 - val_loss: 0.8327 - val_accuracy: 0.6905
Epoch 39/50
44/44 - 5s - loss: 0.3389 - accuracy: 0.8995 - val_loss: 0.8371 - val_accuracy: 0.6934
Epoch 40/50
44/44 - 6s - loss: 0.3292 - accuracy: 0.9013 - val_loss: 0.8427 - val_accuracy: 0.6905
Epoch 41/50
44/44 - 5s - loss: 0.3204 - accuracy: 0.9052 - val_loss: 0.8495 - val_accuracy: 0.6934
Epoch 42/50
44/44 - 4s - loss: 0.3117 - accuracy: 0.9070 - val_loss: 0.8550 - val_accuracy: 0.6934
Epoch 43/50
44/44 - 5s

Epoch 16/50
44/44 - 3s - loss: 1.2823 - accuracy: 0.5120 - val_loss: 1.3761 - val_accuracy: 0.5272
Epoch 17/50
44/44 - 2s - loss: 1.2438 - accuracy: 0.5386 - val_loss: 1.3464 - val_accuracy: 0.5387
Epoch 18/50
44/44 - 3s - loss: 1.1953 - accuracy: 0.5397 - val_loss: 1.2938 - val_accuracy: 0.5501
Epoch 19/50
44/44 - 3s - loss: 1.1628 - accuracy: 0.5605 - val_loss: 1.2880 - val_accuracy: 0.5473
Epoch 20/50
44/44 - 3s - loss: 1.1392 - accuracy: 0.5785 - val_loss: 1.2287 - val_accuracy: 0.6017
Epoch 21/50
44/44 - 3s - loss: 1.1093 - accuracy: 0.5749 - val_loss: 1.2144 - val_accuracy: 0.6017
Epoch 22/50
44/44 - 3s - loss: 1.1011 - accuracy: 0.5928 - val_loss: 1.1952 - val_accuracy: 0.6218
Epoch 23/50
44/44 - 3s - loss: 1.0959 - accuracy: 0.5910 - val_loss: 1.1859 - val_accuracy: 0.6447
Epoch 24/50
44/44 - 3s - loss: 1.0415 - accuracy: 0.6101 - val_loss: 1.1523 - val_accuracy: 0.6504
Epoch 25/50
44/44 - 2s - loss: 1.0452 - accuracy: 0.6068 - val_loss: 1.1431 - val_accuracy: 0.6418
Epoch 26/5

Train for 44 steps, validate for 6 steps
Epoch 1/50
44/44 - 3s - loss: 2.0180 - accuracy: 0.2740 - val_loss: 2.1196 - val_accuracy: 0.2980
Epoch 2/50
44/44 - 3s - loss: 1.8853 - accuracy: 0.3106 - val_loss: 2.0504 - val_accuracy: 0.3123
Epoch 3/50
44/44 - 2s - loss: 1.8385 - accuracy: 0.3142 - val_loss: 1.9981 - val_accuracy: 0.3181
Epoch 4/50
44/44 - 2s - loss: 1.7879 - accuracy: 0.3117 - val_loss: 1.9115 - val_accuracy: 0.3524
Epoch 5/50
44/44 - 2s - loss: 1.7189 - accuracy: 0.3497 - val_loss: 1.8227 - val_accuracy: 0.3696
Epoch 6/50
44/44 - 3s - loss: 1.6578 - accuracy: 0.3688 - val_loss: 1.7551 - val_accuracy: 0.3926
Epoch 7/50
44/44 - 2s - loss: 1.5885 - accuracy: 0.3989 - val_loss: 1.6647 - val_accuracy: 0.4183
Epoch 8/50
44/44 - 2s - loss: 1.5065 - accuracy: 0.4230 - val_loss: 1.5932 - val_accuracy: 0.4499
Epoch 9/50
44/44 - 2s - loss: 1.4698 - accuracy: 0.4470 - val_loss: 1.5461 - val_accuracy: 0.4413
Epoch 10/50
44/44 - 2s - loss: 1.4019 - accuracy: 0.4826 - val_loss: 1.4900 -

44/44 - 7s - loss: 0.3918 - accuracy: 0.8661 - val_loss: 0.8064 - val_accuracy: 0.7249
Epoch 34/50
44/44 - 8s - loss: 0.3725 - accuracy: 0.8754 - val_loss: 0.7813 - val_accuracy: 0.7393
Epoch 35/50
44/44 - 7s - loss: 0.3723 - accuracy: 0.8729 - val_loss: 0.7648 - val_accuracy: 0.7421
Epoch 36/50
44/44 - 7s - loss: 0.3638 - accuracy: 0.8743 - val_loss: 0.7770 - val_accuracy: 0.7536
Epoch 37/50
44/44 - 6s - loss: 0.3529 - accuracy: 0.8833 - val_loss: 0.7519 - val_accuracy: 0.7650
Epoch 38/50
44/44 - 6s - loss: 0.3443 - accuracy: 0.8869 - val_loss: 0.7470 - val_accuracy: 0.7679
Epoch 39/50
44/44 - 7s - loss: 0.3293 - accuracy: 0.8923 - val_loss: 0.7679 - val_accuracy: 0.7650
Epoch 40/50
44/44 - 6s - loss: 0.3274 - accuracy: 0.8923 - val_loss: 0.7414 - val_accuracy: 0.7593
Epoch 41/50
44/44 - 8s - loss: 0.3186 - accuracy: 0.8987 - val_loss: 0.7723 - val_accuracy: 0.7507
Epoch 42/50
44/44 - 6s - loss: 0.3052 - accuracy: 0.8941 - val_loss: 0.7815 - val_accuracy: 0.7708
Epoch 43/50
44/44 - 7s

...

44/44 - 6s - loss: 0.3789 - accuracy: 0.8736 - val_loss: 1.0438 - val_accuracy: 0.7192
Epoch 34/50
44/44 - 8s - loss: 0.3644 - accuracy: 0.8779 - val_loss: 1.0248 - val_accuracy: 0.7221
Epoch 35/50
44/44 - 6s - loss: 0.3600 - accuracy: 0.8772 - val_loss: 1.0145 - val_accuracy: 0.7192
Epoch 36/50
44/44 - 7s - loss: 0.3511 - accuracy: 0.8837 - val_loss: 1.0454 - val_accuracy: 0.7049
Epoch 37/50
44/44 - 7s - loss: 0.3366 - accuracy: 0.8887 - val_loss: 1.0219 - val_accuracy: 0.7221
Epoch 38/50
44/44 - 6s - loss: 0.3243 - accuracy: 0.8898 - val_loss: 1.0010 - val_accuracy: 0.7364
Epoch 39/50
44/44 - 7s - loss: 0.3301 - accuracy: 0.8855 - val_loss: 1.0431 - val_accuracy: 0.7106
Epoch 40/50
44/44 - 7s - loss: 0.3038 - accuracy: 0.8973 - val_loss: 1.0207 - val_accuracy: 0.7192
Epoch 41/50
44/44 - 6s - loss: 0.2950 - accuracy: 0.9005 - val_loss: 1.0462 - val_accuracy: 0.7249
Epoch 42/50
44/44 - 8s - loss: 0.3053 - accuracy: 0.8948 - val_loss: 1.0267 - val_accuracy: 0.7364
Epoch 43/50
44/44 - 7s

Epoch 16/50
44/44 - 3s - loss: 0.5033 - accuracy: 0.8312 - val_loss: 0.8697 - val_accuracy: 0.7249
Epoch 17/50
44/44 - 3s - loss: 0.4898 - accuracy: 0.8395 - val_loss: 0.8756 - val_accuracy: 0.7221
Epoch 18/50
44/44 - 3s - loss: 0.4653 - accuracy: 0.8434 - val_loss: 0.8831 - val_accuracy: 0.7135
Epoch 19/50
44/44 - 3s - loss: 0.4437 - accuracy: 0.8539 - val_loss: 0.8794 - val_accuracy: 0.7106
Epoch 20/50
44/44 - 3s - loss: 0.4326 - accuracy: 0.8557 - val_loss: 0.8915 - val_accuracy: 0.7077
Epoch 21/50
44/44 - 3s - loss: 0.4139 - accuracy: 0.8575 - val_loss: 0.8751 - val_accuracy: 0.7249
Epoch 22/50
44/44 - 3s - loss: 0.3962 - accuracy: 0.8729 - val_loss: 0.8550 - val_accuracy: 0.7335
Epoch 23/50
44/44 - 3s - loss: 0.3849 - accuracy: 0.8715 - val_loss: 0.8571 - val_accuracy: 0.7249
Epoch 24/50
44/44 - 3s - loss: 0.3673 - accuracy: 0.8772 - val_loss: 0.9100 - val_accuracy: 0.7163
Epoch 25/50
44/44 - 3s - loss: 0.3542 - accuracy: 0.8811 - val_loss: 0.8721 - val_accuracy: 0.7221
Epoch 26/5

Train for 44 steps, validate for 6 steps
Epoch 1/50
44/44 - 8s - loss: 1.6815 - accuracy: 0.3975 - val_loss: 1.6188 - val_accuracy: 0.4670
Epoch 2/50
44/44 - 8s - loss: 1.3475 - accuracy: 0.5386 - val_loss: 1.4671 - val_accuracy: 0.5444
Epoch 3/50
44/44 - 8s - loss: 1.2085 - accuracy: 0.5885 - val_loss: 1.3900 - val_accuracy: 0.5645
Epoch 4/50
44/44 - 7s - loss: 1.1258 - accuracy: 0.6151 - val_loss: 1.3459 - val_accuracy: 0.5673
Epoch 5/50
44/44 - 6s - loss: 1.0572 - accuracy: 0.6373 - val_loss: 1.3169 - val_accuracy: 0.5759
Epoch 6/50
44/44 - 8s - loss: 0.9993 - accuracy: 0.6535 - val_loss: 1.2771 - val_accuracy: 0.5989
Epoch 7/50
44/44 - 8s - loss: 0.9503 - accuracy: 0.6740 - val_loss: 1.2660 - val_accuracy: 0.6189
Epoch 8/50
44/44 - 6s - loss: 0.8950 - accuracy: 0.6955 - val_loss: 1.2320 - val_accuracy: 0.6160
Epoch 9/50
44/44 - 8s - loss: 0.8555 - accuracy: 0.7052 - val_loss: 1.2372 - val_accuracy: 0.6189
Epoch 10/50
44/44 - 8s - loss: 0.8103 - accuracy: 0.7153 - val_loss: 1.2025 -

Train for 44 steps, validate for 6 steps
Epoch 1/50
44/44 - 10s - loss: 1.4938 - accuracy: 0.4736 - val_loss: 1.3262 - val_accuracy: 0.5960
Epoch 2/50
44/44 - 6s - loss: 1.1649 - accuracy: 0.5896 - val_loss: 1.2193 - val_accuracy: 0.6504
Epoch 3/50
44/44 - 6s - loss: 1.0626 - accuracy: 0.6302 - val_loss: 1.1666 - val_accuracy: 0.6390
Epoch 4/50
44/44 - 5s - loss: 0.9904 - accuracy: 0.6560 - val_loss: 1.1179 - val_accuracy: 0.6418
Epoch 5/50
44/44 - 6s - loss: 0.9359 - accuracy: 0.6826 - val_loss: 1.1056 - val_accuracy: 0.6476
Epoch 6/50
44/44 - 6s - loss: 0.8803 - accuracy: 0.6952 - val_loss: 1.0832 - val_accuracy: 0.6418
Epoch 7/50
44/44 - 6s - loss: 0.8428 - accuracy: 0.7106 - val_loss: 1.0872 - val_accuracy: 0.6332
Epoch 8/50
44/44 - 5s - loss: 0.7978 - accuracy: 0.7246 - val_loss: 1.0954 - val_accuracy: 0.6304
Epoch 9/50
44/44 - 5s - loss: 0.7747 - accuracy: 0.7350 - val_loss: 1.0975 - val_accuracy: 0.6189
Epoch 10/50
44/44 - 6s - loss: 0.7242 - accuracy: 0.7487 - val_loss: 1.0897 

Train for 44 steps, validate for 6 steps
Epoch 1/50
44/44 - 7s - loss: 1.4684 - accuracy: 0.4765 - val_loss: 1.3233 - val_accuracy: 0.5587
Epoch 2/50
44/44 - 3s - loss: 1.1444 - accuracy: 0.6090 - val_loss: 1.2105 - val_accuracy: 0.5989
Epoch 3/50
44/44 - 4s - loss: 1.0361 - accuracy: 0.6524 - val_loss: 1.1579 - val_accuracy: 0.6103
Epoch 4/50
44/44 - 3s - loss: 0.9704 - accuracy: 0.6657 - val_loss: 1.1278 - val_accuracy: 0.6390
Epoch 5/50
44/44 - 3s - loss: 0.9247 - accuracy: 0.6844 - val_loss: 1.1046 - val_accuracy: 0.6533
Epoch 6/50
44/44 - 3s - loss: 0.8684 - accuracy: 0.7031 - val_loss: 1.0800 - val_accuracy: 0.6504
Epoch 7/50
44/44 - 3s - loss: 0.8275 - accuracy: 0.7206 - val_loss: 1.0523 - val_accuracy: 0.6877
Epoch 8/50
44/44 - 3s - loss: 0.7878 - accuracy: 0.7343 - val_loss: 1.0602 - val_accuracy: 0.6734
Epoch 9/50
44/44 - 3s - loss: 0.7509 - accuracy: 0.7436 - val_loss: 1.0492 - val_accuracy: 0.6648
Epoch 10/50
44/44 - 3s - loss: 0.7111 - accuracy: 0.7569 - val_loss: 1.0763 -

In [ ]:
trial = 10
print(seed)
d_vgg16['seed_value'] = seed
d_resnet50['seed_value'] = seed
d_mobilenetv2['seed_value'] = seed

d_glove['seed_value'] = seed
d_fasttext['seed_value'] = seed

d_glove_vgg16['seed_value'] = seed
d_glove_resnet50['seed_value'] = seed
d_glove_mobilenetv2['seed_value'] = seed

d_fasttext_vgg16['seed_value'] = seed
d_fasttext_resnet50['seed_value'] = seed
d_fasttext_mobilenetv2['seed_value'] = seed

vgg16_result = pd.DataFrame(d_vgg16, index = ["run-"+str(i+1) for i in range(trial)])
resnet50_result = pd.DataFrame(d_resnet50, index = ["run-"+str(i+1) for i in range(trial)])
mobilenetv2_result = pd.DataFrame(d_mobilenetv2, index = ["run-"+str(i+1) for i in range(trial)])

glove_result = pd.DataFrame(d_glove, index = ["run-"+str(i+1) for i in range(trial)])
fasttext_result = pd.DataFrame(d_fasttext, index = ["run-"+str(i+1) for i in range(trial)])

glove_vgg16_result = pd.DataFrame(d_glove_vgg16, index = ["run-"+str(i+1) for i in range(trial)])
glove_resnet50_result = pd.DataFrame(d_glove_resnet50, index = ["run-"+str(i+1) for i in range(trial)])
glove_mobilenetv2_result = pd.DataFrame(d_glove_mobilenetv2, index = ["run-"+str(i+1) for i in range(trial)])

fasttext_vgg16_result = pd.DataFrame(d_fasttext_vgg16, index = ["run-"+str(i+1) for i in range(trial)])
fasttext_resnet50_result = pd.DataFrame(d_fasttext_resnet50, index = ["run-"+str(i+1) for i in range(trial)])
fasttext_mobilenetv2_result = pd.DataFrame(d_fasttext_mobilenetv2, index = ["run-"+str(i+1) for i in range(trial)])


[69, 53, 90, 29, 12, 47, 79, 8, 43, 82]


In [ ]:
vgg16_result.to_csv("/home/PiyaliBhunia/Desktop/GPU_shared_data/new_result/vgg16_result.csv")
resnet50_result.to_csv("/home/PiyaliBhunia/Desktop/GPU_shared_data/new_result/resnet50_result.csv")
mobilenetv2_result.to_csv("/home/PiyaliBhunia/Desktop/GPU_shared_data/new_result/mobilenetv2_result.csv")

glove_result.to_csv("/home/PiyaliBhunia/Desktop/GPU_shared_data/new_result/glove_result.csv")
fasttext_result.to_csv("/home/PiyaliBhunia/Desktop/GPU_shared_data/new_result/fasttext_result.csv")

glove_vgg16_result.to_csv("/home/PiyaliBhunia/Desktop/GPU_shared_data/new_result/Multimodal_result_glove_and_vgg16.csv")
glove_resnet50_result.to_csv("/home/PiyaliBhunia/Desktop/GPU_shared_data/new_result/Multimodal_result_glove_and_resnet50.csv")
glove_mobilenetv2_result.to_csv("/home/PiyaliBhunia/Desktop/GPU_shared_data/new_result/Multimodal_result_glove_and_mobilenetv2.csv")

fasttext_vgg16_result.to_csv("/home/PiyaliBhunia/Desktop/GPU_shared_data/new_result/Multimodal_result_fasttext_and_vgg16.csv")
fasttext_resnet50_result.to_csv("/home/PiyaliBhunia/Desktop/GPU_shared_data/new_result/Multimodal_result_fasttext_and_resnet50.csv")
fasttext_mobilenetv2_result.to_csv("/home/PiyaliBhunia/Desktop/GPU_shared_data/new_result/Multimodal_result_fasttext_and_mobilenetv2.csv")